<a href="https://colab.research.google.com/github/fjadidi2001/DecompDenoise/blob/main/DD_Feb12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
from scipy.fft import fft2, ifft2

def decompose_and_denoise(image_path, threshold=30):
    """
    Implement noise reduction using frequency domain decomposition.

    Parameters:
    image_path (str): Path to the input image
    threshold (float): Threshold for frequency filtering

    Returns:
    numpy.ndarray: Denoised image
    """
    # Read the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Could not read image: {image_path}")

    # Convert to float for better precision
    img_float = img.astype(np.float32) / 255.0

    # Apply 2D Fourier Transform
    f_transform = fft2(img_float)
    f_shift = np.fft.fftshift(f_transform)

    # Create frequency mask
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.ones((rows, cols), np.uint8)

    # Apply frequency threshold
    magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)
    mask[magnitude_spectrum > threshold] = 0

    # Apply mask and inverse transform
    f_shift_filtered = f_shift * mask
    f_ishift = np.fft.ifftshift(f_shift_filtered)
    img_back = np.abs(ifft2(f_ishift))

    # Normalize and convert back to uint8
    img_denoised = np.clip(img_back * 255, 0, 255).astype(np.uint8)

    return img_denoised

def analyze_noise_reduction(original, denoised):
    """
    Analyze the effectiveness of noise reduction.

    Parameters:
    original (numpy.ndarray): Original image
    denoised (numpy.ndarray): Denoised image

    Returns:
    dict: Dictionary containing analysis metrics
    """
    mse = np.mean((original - denoised) ** 2)
    if mse == 0:
        psnr = float('inf')
    else:
        psnr = 20 * np.log10(255.0 / np.sqrt(mse))

    return {
        'MSE': mse,
        'PSNR': psnr,
        'Mean_Difference': np.mean(np.abs(original - denoised))
    }

# Example usage
if __name__ == "__main__":
    image_path = "test_001.png"

    try:
        # Perform denoising
        denoised_image = decompose_and_denoise(image_path)

        # Read original image for comparison
        original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Analyze results
        metrics = analyze_noise_reduction(original_image, denoised_image)

        # Save results
        cv2.imwrite("denoised_result.png", denoised_image)

        print("Denoising completed successfully")
        print("Analysis metrics:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.2f}")

    except Exception as e:
        print(f"Error: {str(e)}")